In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score, roc_curve
from nltk.stem.snowball import EnglishStemmer

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 500)

In [2]:
from sklearn.neighbors import NearestNeighbors

recommender,   
user lists ONE ingredient   
returns cocktail with filter on ingredient.
vecto user input.  
+ returns 5 cocktail from different clusters (10-15 cocktail par cluster, cluster_id, 100+ clusters) 
filter on user ingredient, drop duplicates from clusters (select one only)  
from this cocktail, find other cocktails containing a common ingredient (Kmeans)

_______________________________

AI generated cocktails:  
table avec index = ingrdeient, col= cocktail name
(PCA)
user input = ingredient  
algo returns other ingredients that "go together" (Knearest neighbors in the table), those seen often together in cocktails.  
  
user can re-input a suggested ingredient, iterates on the process (sum or avg)  
stores the avg/count of the inputted ingredients, and reruns 

For one-hot encoded or count-encoded data (e.g. country, region, or text), you should normalize by row (i.e. individual), before concatenating with the other columns. You can use for that https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html

In [ ]:
df_pivoted = pd.pivot_table(cocktail_ingredient_table, index='ingredient_id', columns='cocktail_id', aggfunc='count')
df_pivoted.fillna(0, inplace= True)

In [ ]:
ingredient_table # pour chaque cocktail, concat tt les ingredients DU cocktail (' ing1 ing2 ing3 ing4')
#  ensuite utiliser countvextorizer qui va découper TOUS les mots (genre 'orange =1', 'juice=1')

In [ ]:
#  concat tables to have a table with 
index = ctl_name
col = ingredients (words)

In [43]:
ctl_name = pd.read_csv('./CSVs/ctl_name_id.csv', sep=',', index_col=[0])
ctl_ingredient_words = pd.read_csv('./CSVs/ingredient_table.csv', sep=',',index_col=[0])
ctl_ing_id = pd.read_csv('./CSVs/cocktail_ingredient_table.csv', sep=',')

In [45]:
ctl_ing_id.drop(columns='typing.Literal[<no_default>]', inplace=True)

In [46]:
ctl_ing_id

,cocktail_id,ingredient_id,quantity
0,0,461,1 oz
1,0,265,.5 oz
2,0,263,.25 oz
3,0,257,1 oz
4,0,523,1 tsp
...,...,...,...
18690,4652,965,3 oz
18691,4652,381,1 oz
18692,4675,88,3 oz
18693,4675,787,2 oz


In [23]:
ctl_ingredient_words.dropna(inplace=True)

In [26]:
ctl_ingredient_words

,ingredient_id,ingredients
1,1,Iced Tea
2,2,Italian Amaro
3,3,Simple Syrup
4,4,Triple Sec
5,5,Scotch Liqueur
...,...,...
982,982,Peppermint and Hot Cinnamon Schnapps
983,983,(1 ml) Vodka
984,984,Mixed Fruit
985,985,Mint Schnapps


In [27]:
ctl_name

,cocktail_id,name
0,0,American Beauty Cocktail
1,1,Azzuro
2,2,Apple Fairy
3,3,Black Rose Bacardi
4,4,Blue Latvian
...,...,...
4685,5111,Brown Hen
4686,5112,Bosom Caresser
4687,5113,Brandy Water Press
4688,5114,Big Dipper


In [54]:
df_temp_merge = pd.merge(ctl_ing_id,ctl_ingredient_words, on='ingredient_id')

In [55]:
df_temp_merge.drop(columns='quantity', inplace=True)

In [56]:
df_merged = pd.merge(df_temp_merge,ctl_name,on='cocktail_id' )

In [59]:
df_merged

,cocktail_id,ingredient_id,ingredients,name
0,0,461,Brandy,American Beauty Cocktail
1,0,265,Dry Vermouth,American Beauty Cocktail
2,0,263,White Creme De Menthe,American Beauty Cocktail
3,0,257,Orange Juice,American Beauty Cocktail
4,0,523,Grenadine Syrup,American Beauty Cocktail
...,...,...,...,...
18690,4430,475,(4500 ml) Watermelon,Vodka infused Watermelon
18691,4430,983,(1 ml) Vodka,Vodka infused Watermelon
18692,4675,88,(90 ml) Prosecco,Aperol Spritz
18693,4675,787,(60 ml) Aperol,Aperol Spritz


In [61]:
df_ing_ctl_matrix = df_merged.drop(columns=['cocktail_id','ingredient_id'])

In [62]:
df_ing_ctl_matrix

,ingredients,name
0,Brandy,American Beauty Cocktail
1,Dry Vermouth,American Beauty Cocktail
2,White Creme De Menthe,American Beauty Cocktail
3,Orange Juice,American Beauty Cocktail
4,Grenadine Syrup,American Beauty Cocktail
...,...,...
18690,(4500 ml) Watermelon,Vodka infused Watermelon
18691,(1 ml) Vodka,Vodka infused Watermelon
18692,(90 ml) Prosecco,Aperol Spritz
18693,(60 ml) Aperol,Aperol Spritz


In [64]:
df_pivoted = pd.pivot_table(df_ing_ctl_matrix, index='name', columns='ingredients', aggfunc='count')
df_pivoted.fillna(0, inplace= True)

In [ ]:
df_ctl_info = pd.read_csv('./CSVs/ctl_name_id.csv', sep=',', index_col=[0])

In [79]:
pivot_df

,0
name,
American Beauty Cocktail,NaN
American Beauty Cocktail,NaN
American Beauty Cocktail,NaN
American Beauty Cocktail,NaN
American Beauty Cocktail,NaN
...,...
NaN,1.0
NaN,4.0
NaN,1.0


### Load the data

In [7]:
ctl_info = pd.read_csv('./CSVs/unique_cocktail_info.csv', sep=',')

In [9]:
ctl_info.rename(columns={'Unnamed: 0':'cocktail_id'})

,cocktail_id,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,0,American Beauty Cocktail,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz: White Creme De Menthe | 1 oz: Orange Juice | 1 tsp: Grenadine Syrup | 1 oz: Red Port,Calories: 185kcal | Carbohydrates: 14g | Protein: 0.3g | Fat: 0.1g | Saturated Fat: 0.01g | Polyunsaturated Fat: 0.02g | Monounsaturated Fat: 0.01g | Sodium: 6mg | Potassium: 89mg | Fiber: 0.1g | Sugar: 10g | Vitamin A: 50IU | Vitamin C: 20.6mg,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1,1,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,2,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,3,Black Rose Bacardi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake or stir. pour it into a coctail glass. add some crushed ice and serve.,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,4,Blue Latvian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Jack Daniel`s whiskey into a mixing glass. Add blue raspberry juice until the mixture is more blue than brown. Finish with lemon-lime soda until the mixture is fizzing. Serve in a collins glass.,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
...,...,...,...,...,...,...,...
4685,5111,Brown Hen,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Fill glass with ice. then Vodka. then bitters. Stir.,| 2 oz: Vodka | 1 tsp tsp: Bitters,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4686,5112,Bosom Caresser,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",In a mixing glass half-filled with crushed ice. combine all of the ingredients. Stir well. Strain into a cocktail glass.,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madeira | .5 oz: Triple Sec,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4687,511

In [8]:
ctl_info.head()

,Unnamed: 0,Name,Description,Recipe,Ingredients,Nutrition Facts,Video Link
0,0,American Beauty Cocktail,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Woops... We couldn't retrieve the exact recipe... It's trial & error time! Just a little more fun before enjoying a nice drink!,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz: White Creme De Menthe | 1 oz: Orange Juice | 1 tsp: Grenadine Syrup | 1 oz: Red Port,Calories: 185kcal | Carbohydrates: 14g | Protein: 0.3g | Fat: 0.1g | Saturated Fat: 0.01g | Polyunsaturated Fat: 0.02g | Monounsaturated Fat: 0.01g | Sodium: 6mg | Potassium: 89mg | Fiber: 0.1g | Sugar: 10g | Vitamin A: 50IU | Vitamin C: 20.6mg,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
1,1,Azzuro,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake and strain into an ice-filled collins glass. and garnish with fruit.,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Curacao Liqueur | 1 oz: Banana Liqueur | 2 oz: Passion Fruit Juice | 2 oz: Apple Juice | 1 tsp: Egg,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
2,2,Apple Fairy,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",For this recipe. make some Juice - Apple Juice ice cubes before you start. Then add 3 Juice - Apple Juice cubes to a cocktail glass. Add absinthe and apple Vodka. and fill with cider.,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodka | 3 oz: Apple Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
3,3,Black Rose Bacardi,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Shake or stir. pour it into a coctail glass. add some crushed ice and serve.,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz: Lime Juice,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!
4,4,Blue Latvian,"Unfortunately, we have no description for this drink... You'll have to describe it yourself!",Pour the Jack Daniel`s whiskey into a mixing glass. Add blue raspberry juice until the mixture is more blue than brown. Finish with lemon-lime soda until the mixture is fizzing. Serve in a collins glass.,| 1 oz: Bourbon Whiskey |: Raspberry Juice |: Lemon-Lime Soda,There doesn't seem to be any nutritional facts associated to this cocktail. But we know that's not why you're here...,There doesn't seem to be an instructional video for this cocktail. Why not make the tutorial yourself!


In [8]:
ctl_ingredients = ctl_df['Ingredients']

In [10]:
ctl_ingredients = pd.DataFrame(ctl_ingredients)
ctl_ingredients

,Ingredients
0,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz...
1,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Cur...
2,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodk...
3,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz:...
4,| 1 oz: Bourbon Whiskey |: Raspberry Juice |:...
...,...
5111,| 2 oz: Vodka | 1 tsp tsp: Bitters
5112,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madei...
5113,| 1.5 oz: Brandy | .5 oz: Tonic Water |: Water
5114,| 1 oz: Dark Rum | 1 oz: Brandy | 1 tbsp: Lim...


## 1: remove all but the ingredient name using regex  
## 2: split ingredients into columns

#### 1: remove all but the ingredient name

In [11]:
ctl_ingredients

,Ingredients
0,| 1 oz: Brandy | .5 oz: Dry Vermouth | .25 oz...
1,| 1 oz: Exotic-Fruit Liqueur | 1 oz: Blue Cur...
2,| 2/3 oz: Herbal Liqueur | 2/3 oz: Apple Vodk...
3,|: Ice | 1 oz: White Rum | 2 oz: Cola | 2 oz:...
4,| 1 oz: Bourbon Whiskey |: Raspberry Juice |:...
...,...
5111,| 2 oz: Vodka | 1 tsp tsp: Bitters
5112,|: Crushed Ice | 1.5 oz: Brandy | 1 oz: Madei...
5113,| 1.5 oz: Brandy | .5 oz: Tonic Water |: Water
5114,| 1 oz: Dark Rum | 1 oz: Brandy | 1 tbsp: Lim...


In [ ]:
for i in ctl_ingredients['Ingredients']:
    i.replace(everything from "|" to the next uppercase, by ' ')

In [19]:
ctl_ingredients_clean = ctl_ingredients['Ingredients'].str.extractall(r"([A-Z].*?\w+(?=[A-Z]))")

In [17]:
ctl_ingredients_clean

0
     match                            
62   0      Banana Schnapps | .5 oz: O
147  0                   L: Gin | 10 m
     1            L: Triple Sec | 20 m
     2         L: Coconut Cream | 10 m
     3        L: Banana Liqueur | 10 m
...                                ...
5029 2          L: Orange Juice | 60 m
     3       L: Cranberry Juice | 25 m
     4         L: Cherry Brandy | 60 m
     5           L: Energy Soda | 25 m
5072 0           Apple Juice | 1 oz: O

[186 rows x 1 columns]